In [1]:
# !pip install selenium

In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd

In [2]:
driver = webdriver.Chrome(r"C:\Users\hp\chromedriver.exe")

In [3]:
url="https://www.volunteertoronto.ca/networking/search_results.asp?__ASPVIEWSTATE=e08802521ff4e66a05946546c72bd4b7b6a8c4ec67aebd98e7ba4a7c33ee88cf0559ad859e96cd53ec831d5de52500452ccfd7f3294ba660a4cdc177a9dfa754263a0a863fdfc220d3c3b73747f7dd9d030f0626f5b240ae707c2a1deade68d6fb6b2e72d5abfe4a74913dfdd6a9a5602ffb74bb15725854b6e9fcde32138d1be5ae35eda9bcacbeb5dadcad15b3253ccbeea14d9cffca60a50352630d07b3276f607c057e845898339fda1ea2222b7d2bced390053cee7e48aa6c21d699cf69d959f6d7ce4e0d10"
driver.get(url)

In [6]:
prefix_url = 'https://www.volunteertoronto.ca/'
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

In [7]:
Pages=[url]
page=soup.find_all('div', attrs={'class':'pager'})
for p in page:
    pp=p.find_all('a')
    for l in pp:
        w=l.get("href")
        if prefix_url+w not in Pages:
            Pages.append(prefix_url+w)

In [308]:
len(Pages)

6

## Functions

In [11]:
def get_data(url):
    data=[]
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    table = soup.find('table', attrs={'class':'datagrid1'})
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    return table, data

In [17]:
def get_maininfo(data):
    for i in range(1,len(data)):
        if len(data[i]) == 4:
            x = data[i][0].split('\r\n\t\t\t\t')
            if len(x)>1:
                Location.append(x[1])
            else:
                Location.append('nan')
            Position.append(x[0])
            Primary_Category.append(data[i][1])
            Organization.append(data[i][2])
            Date_Posted.append(data[i][3])
        # No organization
        elif len(data[i]) == 3:
            x = data[i][0].split('\r\n\t\t\t\t')
            if len(x)>1:
                Location.append(x[1])
            else:
                Location.append('nan')
            Position.append(x[0])
            Primary_Category.append(data[i][1])
            Organization.append("nan")
            Date_Posted.append(data[i][2])
    return Position, Primary_Category, Organization, Date_Posted

In [13]:
def get_websites(table):
    webs=table.findAll('a')
    href=[]
    prefix_url="https://www.volunteertoronto.ca/networking/"
    for link in webs:
        if len(link.get("href")) > 2 and len(link.get("href")) < 100:
            href.append(link.get("href"))
    for i in range(len(href)):
        Websites.append(prefix_url+href[i])
    return Websites

In [14]:
def get_description(websites):
    for link in Websites:
        html_content = requests.get(link).text
        soup = BeautifulSoup(html_content, "lxml")
        des=soup.find_all('tr')
        idx_des = [i for i in range(len(des)) if des[i].find('b',string='Description & Details') != None]
        idx=des[idx_des[-1]+1]
        Description.append(idx.get_text())
    return Description

### Get Info

In [18]:
Position=[]
Location=[]
Primary_Category=[]
Organization=[]
Date_Posted=[]
Websites=[]
Description=[]

In [19]:
for link in Pages:    
    table, data = get_data(link)
    Position, Primary_Category, Organization, Date_Posted = get_maininfo(data)
    Websites = get_websites(table)
Description = get_description(Websites)

https://www.volunteertoronto.ca/networking/search_results.asp?__ASPVIEWSTATE=e08802521ff4e66a05946546c72bd4b7b6a8c4ec67aebd98e7ba4a7c33ee88cf0559ad859e96cd53ec831d5de52500452ccfd7f3294ba660a4cdc177a9dfa754263a0a863fdfc220d3c3b73747f7dd9d030f0626f5b240ae707c2a1deade68d6fb6b2e72d5abfe4a74913dfdd6a9a5602ffb74bb15725854b6e9fcde32138d1be5ae35eda9bcacbeb5dadcad15b3253ccbeea14d9cffca60a50352630d07b3276f607c057e845898339fda1ea2222b7d2bced390053cee7e48aa6c21d699cf69d959f6d7ce4e0d10
https://www.volunteertoronto.ca//networking/search_results.asp?__ASPVIEWSTATE=e08802521ff4e66a05946546c72bd4b7b6a8c4ec67aebd98e7ba4a7c33ee88cf0559ad859e96cd53ec831d5de52500452ccfd7f3294ba660a4cdc177a9dfa754263a0a863fdfc220d3c3b73747f7dd9d030f0626f5b240ae707c2a1deade68d6fb6b2e72d5abfe4a74913dfdd6a9a5602ffb74bb15725854b6e9fcde32138d1be5ae35eda9bcacbeb5dadcad15b3253ccbeea14d9cffca60a50352630d07b3276f607c057e845898339fda1ea2222b7d2bced390053cee7e48aa6c21d699cf69d959f6d7ce4e0d10&DGPCrSrt=&DGPCrPg=2
https://www.volunteerto

In [20]:
print("Position:{}".format(len(Position)))
print("Primary_Category:{}".format(len(Primary_Category)))
print("Organization:{}".format(len(Organization)))
print("Date_Posted:{}".format(len(Date_Posted)))
print("Websites:{}".format(len(Websites)))
print("Description:{}".format(len(Description)))

Position:252
Primary_Category:252
Organization:252
Date_Posted:252
Websites:252
Description:252


### Output CSV file

In [21]:
df = pd.DataFrame({'Position':Position,'Location':Location, 
                   'Category':Primary_Category,
                 'Organization':Organization,
                 'Date Posted':Date_Posted,
                  "Websites": Websites,
                  "Description": Description}) 

In [22]:
df.to_csv('toronto_volunteer.csv', index=False, encoding='utf-8_sig')

In [23]:
df

,Position,Location,Category,Organization,Date Posted,Websites,Description
0,"Co-Director, Sponsorship","Toronto, Ontario","Boards, Committees, and Leadership",Endeavour Volunteer Consulting for Non-Profits,2020-07-04,https://www.volunteertoronto.ca/networking/app...,\n\nROLE\n \nEndeavour is looking for a volunt...
1,COVID-19: Driver,"Etobicoke, Toronto, North York, and Scarborough,",COVID-19 Response,CEE Centre for Young Black Professionals,2020-07-03,https://www.volunteertoronto.ca/networking/app...,"\n\nThe CEE (Careers, Education, Empowerment) ..."
2,Advocacy Volunteer,Ontario,"Advocacy, Policy, and Research",Canadian Women's Chamber of Commerce,2020-07-03,https://www.volunteertoronto.ca/networking/app...,\n\n\n \nAbout Us\n \nThe Canadian Women’s Cha...
3,YouTube Community Manager,"Toronto, Ontario","Marketing, Communications, and Fundraising",Single Mothers in Progress,2020-07-03,https://www.volunteertoronto.ca/networking/app...,\n\n \nWe are looking for a YouTube Community ...
4,Writer,"Toronto, Ontario","Marketing, Communications, and Fundraising",Single Mothers in Progress,2020-07-03,https://www.volunteertoronto.ca/networking/app...,\n\n \nWe are looking for a Writer to join our...
5,Website Designer,"Toronto, Ontario","Marketing, Communications, and Fundraising",Single Mothers in Progress,2020-07-03,https://www.volunteertoronto.ca/networking/app...,\n\n \nWe are looking for a Website Designer t...
6,Web Developer,"Toronto, Ontario","Marketing, Communications, and Fundraising",Single Mothers in Progress,2020-07-03,https://www.volunteertoronto.ca/networking/app...,\n\nWe are looking for Web Developers who will...
7,Volunteer Coordinator,"Toronto, Ontario",Administration and Customer Service,Single Mothers in Progress,2020-07-03,https://www.volunteertoronto.ca/networking/app...,\n\n \nVolunteer coordinators recruit voluntee...
8,Video Editor,"Toronto, Ontario","Marketing, Communications, and Fundraising",Single Mothers in Progress,2020-07-03,https://www.volunteertoronto.ca/networking/app...,"\n\n \nUltimately, as a film and video editor,..."
9,Technical support representative,"Toronto, Ontario",IT Support,Single Mothers in Progress,2020-07-03,https://www.volunteertoronto.ca/networking/app...,\n\n \nYou will be supporting end users during...
